In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import os
import numpy as np

class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                # Image labels. 0-9
                img_labels = y_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # The labels of the digits that the generator tries to create an
                # image representation of
                sampled_labels = np.random.randint(0, 10, (batch_size, 1))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, sampled_labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, [valid, img_labels])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss[0]))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)

    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_acgan'):
            os.mkdir('images_acgan')
        fig.savefig("images_acgan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    acgan = ACGAN()
    acgan.train(epochs=30, batch_size=64, sample_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 8, 8, 32)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 3.372124, acc.: 37.50%, op_acc: 7.03%] [G loss: 3.566040]
epoch:0 step:2 [D loss: 3.387449, acc.: 51.56%, op_acc: 6.25%] [G loss: 3.379170]
epoch:0 step:3 [D loss: 3.312998, acc.: 42.19%, op_acc: 14.84%] [G loss: 3.243592]
epoch:0 step:4 [D loss: 3.283644, acc.: 49.22%, op_acc: 12.50%] [G loss: 3.536668]
epoch:0 step:5 [D loss: 3.264128, acc.: 52.34%, op_acc: 10.94%] [G loss: 3.168322]
epoch:0 step:6 [D loss: 3.158976, acc.: 64.06%, op_acc: 14.84%] [G loss: 3.433722]
epoch:0 step:7 [D loss: 3.129742, acc.: 55.47%, op_acc: 14.84%] [G loss: 3.289508]
epoch:0 step:8 [D loss: 3.048900, acc.: 56.25%, op_acc: 14.84%] [G loss: 3.299774]
epoch:0 step:9 [D loss: 3.050465, acc.: 58.59%, op_acc: 14.06%] [G loss: 3.281158]
epoch:0 step:10 [D loss: 3.045772, acc.: 68.75%, op_acc: 17.97%] [G loss: 3.359898]
epoch:0 step:11 [D loss: 3.081340, acc.: 64.06%, op_acc: 10.94%] [G loss: 3.260174]
epoch:0 step:12 [D loss: 3.110360, acc.: 55.47%, op_acc: 10.94%] [G loss: 3.415531]
epo

epoch:0 step:102 [D loss: 1.983935, acc.: 75.78%, op_acc: 50.78%] [G loss: 3.236505]
epoch:0 step:103 [D loss: 2.060281, acc.: 79.69%, op_acc: 47.66%] [G loss: 3.422309]
epoch:0 step:104 [D loss: 2.203026, acc.: 67.19%, op_acc: 48.44%] [G loss: 3.413943]
epoch:0 step:105 [D loss: 2.102741, acc.: 78.91%, op_acc: 45.31%] [G loss: 3.257236]
epoch:0 step:106 [D loss: 2.156470, acc.: 82.03%, op_acc: 42.97%] [G loss: 3.235466]
epoch:0 step:107 [D loss: 2.521406, acc.: 58.59%, op_acc: 35.16%] [G loss: 3.482846]
epoch:0 step:108 [D loss: 2.449914, acc.: 37.50%, op_acc: 50.00%] [G loss: 3.071248]
epoch:0 step:109 [D loss: 2.570139, acc.: 42.19%, op_acc: 41.41%] [G loss: 2.910979]
epoch:0 step:110 [D loss: 2.617007, acc.: 41.41%, op_acc: 42.97%] [G loss: 3.010285]
epoch:0 step:111 [D loss: 2.339925, acc.: 50.00%, op_acc: 42.97%] [G loss: 2.893893]
epoch:0 step:112 [D loss: 2.474370, acc.: 50.78%, op_acc: 40.62%] [G loss: 3.097600]
epoch:0 step:113 [D loss: 2.642707, acc.: 41.41%, op_acc: 42.97%]

epoch:0 step:199 [D loss: 1.465380, acc.: 94.53%, op_acc: 53.12%] [G loss: 4.453824]
epoch:0 step:200 [D loss: 1.545811, acc.: 96.88%, op_acc: 50.00%] [G loss: 4.505557]
epoch:0 step:201 [D loss: 1.275523, acc.: 97.66%, op_acc: 53.91%] [G loss: 4.451395]
epoch:0 step:202 [D loss: 1.483195, acc.: 96.88%, op_acc: 46.88%] [G loss: 4.489855]
epoch:0 step:203 [D loss: 1.506531, acc.: 99.22%, op_acc: 49.22%] [G loss: 4.746838]
epoch:0 step:204 [D loss: 1.566162, acc.: 98.44%, op_acc: 46.09%] [G loss: 4.687474]
epoch:0 step:205 [D loss: 1.521831, acc.: 97.66%, op_acc: 51.56%] [G loss: 4.617021]
epoch:0 step:206 [D loss: 1.482715, acc.: 97.66%, op_acc: 49.22%] [G loss: 4.773921]
epoch:0 step:207 [D loss: 1.249927, acc.: 100.00%, op_acc: 56.25%] [G loss: 4.776088]
epoch:0 step:208 [D loss: 1.230799, acc.: 100.00%, op_acc: 60.94%] [G loss: 4.993134]
epoch:0 step:209 [D loss: 1.418640, acc.: 99.22%, op_acc: 53.91%] [G loss: 4.723735]
epoch:0 step:210 [D loss: 1.282031, acc.: 100.00%, op_acc: 57.0

...

epoch:29 step:27704 [D loss: 0.705002, acc.: 50.78%, op_acc: 100.00%] [G loss: 0.721470]
epoch:29 step:27705 [D loss: 0.713467, acc.: 50.00%, op_acc: 99.22%] [G loss: 0.704874]
epoch:29 step:27706 [D loss: 0.718039, acc.: 46.09%, op_acc: 99.22%] [G loss: 0.723388]
epoch:29 step:27707 [D loss: 0.711821, acc.: 49.22%, op_acc: 100.00%] [G loss: 0.689643]
epoch:29 step:27708 [D loss: 0.710932, acc.: 49.22%, op_acc: 100.00%] [G loss: 0.692032]
epoch:29 step:27709 [D loss: 0.708776, acc.: 47.66%, op_acc: 99.22%] [G loss: 0.709106]
epoch:29 step:27710 [D loss: 0.699415, acc.: 56.25%, op_acc: 100.00%] [G loss: 0.699921]
epoch:29 step:27711 [D loss: 0.757465, acc.: 51.56%, op_acc: 98.44%] [G loss: 0.775939]
epoch:29 step:27712 [D loss: 0.719959, acc.: 46.09%, op_acc: 99.22%] [G loss: 0.693815]
epoch:29 step:27713 [D loss: 0.746264, acc.: 53.91%, op_acc: 98.44%] [G loss: 0.727966]
epoch:29 step:27714 [D loss: 0.729073, acc.: 43.75%, op_acc: 99.22%] [G loss: 0.696695]
epoch:29 step:27715 [D loss:

epoch:29 step:27798 [D loss: 0.706538, acc.: 52.34%, op_acc: 99.22%] [G loss: 0.684127]
epoch:29 step:27799 [D loss: 0.706928, acc.: 54.69%, op_acc: 99.22%] [G loss: 0.732818]
epoch:29 step:27800 [D loss: 0.691522, acc.: 56.25%, op_acc: 100.00%] [G loss: 0.719638]
epoch:29 step:27801 [D loss: 0.784300, acc.: 44.53%, op_acc: 99.22%] [G loss: 0.707526]
epoch:29 step:27802 [D loss: 0.684586, acc.: 56.25%, op_acc: 99.22%] [G loss: 0.695501]
epoch:29 step:27803 [D loss: 0.697375, acc.: 50.78%, op_acc: 99.22%] [G loss: 0.670778]
epoch:29 step:27804 [D loss: 0.728742, acc.: 46.09%, op_acc: 99.22%] [G loss: 0.704103]
epoch:29 step:27805 [D loss: 0.711504, acc.: 48.44%, op_acc: 99.22%] [G loss: 0.681851]
epoch:29 step:27806 [D loss: 0.753242, acc.: 48.44%, op_acc: 99.22%] [G loss: 0.724064]
epoch:29 step:27807 [D loss: 0.693905, acc.: 53.12%, op_acc: 100.00%] [G loss: 0.724726]
epoch:29 step:27808 [D loss: 0.718756, acc.: 51.56%, op_acc: 99.22%] [G loss: 0.689279]
epoch:29 step:27809 [D loss: 0

epoch:29 step:27893 [D loss: 0.778445, acc.: 46.09%, op_acc: 96.88%] [G loss: 0.730736]
epoch:29 step:27894 [D loss: 0.781791, acc.: 38.28%, op_acc: 98.44%] [G loss: 0.678077]
epoch:29 step:27895 [D loss: 0.693472, acc.: 51.56%, op_acc: 100.00%] [G loss: 0.707615]
epoch:29 step:27896 [D loss: 0.744510, acc.: 43.75%, op_acc: 97.66%] [G loss: 0.721836]
epoch:29 step:27897 [D loss: 0.765703, acc.: 53.12%, op_acc: 96.88%] [G loss: 0.714540]
epoch:29 step:27898 [D loss: 0.747093, acc.: 44.53%, op_acc: 98.44%] [G loss: 0.717139]
epoch:29 step:27899 [D loss: 0.782159, acc.: 44.53%, op_acc: 98.44%] [G loss: 0.708372]
epoch:29 step:27900 [D loss: 0.716084, acc.: 50.00%, op_acc: 99.22%] [G loss: 0.742021]
epoch:29 step:27901 [D loss: 0.702497, acc.: 47.66%, op_acc: 100.00%] [G loss: 0.723709]
epoch:29 step:27902 [D loss: 0.703009, acc.: 47.66%, op_acc: 100.00%] [G loss: 0.704539]
epoch:29 step:27903 [D loss: 0.701801, acc.: 49.22%, op_acc: 100.00%] [G loss: 0.713081]
epoch:29 step:27904 [D loss:

epoch:29 step:27989 [D loss: 0.722581, acc.: 63.28%, op_acc: 96.88%] [G loss: 0.738761]
epoch:29 step:27990 [D loss: 0.728204, acc.: 52.34%, op_acc: 99.22%] [G loss: 0.717719]
epoch:29 step:27991 [D loss: 0.708637, acc.: 46.09%, op_acc: 100.00%] [G loss: 0.732301]
epoch:29 step:27992 [D loss: 0.716908, acc.: 49.22%, op_acc: 100.00%] [G loss: 0.715405]
epoch:29 step:27993 [D loss: 0.724826, acc.: 50.78%, op_acc: 98.44%] [G loss: 0.712510]
epoch:29 step:27994 [D loss: 0.712388, acc.: 51.56%, op_acc: 99.22%] [G loss: 0.707710]
epoch:29 step:27995 [D loss: 0.709281, acc.: 41.41%, op_acc: 100.00%] [G loss: 0.701723]
epoch:29 step:27996 [D loss: 0.694865, acc.: 51.56%, op_acc: 100.00%] [G loss: 0.716065]
epoch:29 step:27997 [D loss: 0.752602, acc.: 45.31%, op_acc: 98.44%] [G loss: 0.695550]
epoch:29 step:27998 [D loss: 0.716600, acc.: 46.88%, op_acc: 100.00%] [G loss: 0.712045]
epoch:29 step:27999 [D loss: 0.788729, acc.: 49.22%, op_acc: 96.88%] [G loss: 0.715448]
epoch:29 step:28000 [D loss

epoch:29 step:28084 [D loss: 0.706959, acc.: 46.09%, op_acc: 100.00%] [G loss: 0.720408]
epoch:29 step:28085 [D loss: 0.680125, acc.: 58.59%, op_acc: 100.00%] [G loss: 0.716742]
epoch:29 step:28086 [D loss: 0.703236, acc.: 50.78%, op_acc: 100.00%] [G loss: 0.747576]
epoch:29 step:28087 [D loss: 0.706788, acc.: 48.44%, op_acc: 100.00%] [G loss: 0.724576]
epoch:29 step:28088 [D loss: 0.718169, acc.: 49.22%, op_acc: 100.00%] [G loss: 0.709660]
epoch:29 step:28089 [D loss: 0.714046, acc.: 46.88%, op_acc: 100.00%] [G loss: 0.716084]
epoch:29 step:28090 [D loss: 0.721327, acc.: 44.53%, op_acc: 100.00%] [G loss: 0.708329]
epoch:29 step:28091 [D loss: 0.691779, acc.: 52.34%, op_acc: 100.00%] [G loss: 0.720168]
epoch:29 step:28092 [D loss: 0.727267, acc.: 59.38%, op_acc: 97.66%] [G loss: 0.732182]
epoch:29 step:28093 [D loss: 0.765941, acc.: 48.44%, op_acc: 98.44%] [G loss: 0.717250]
epoch:29 step:28094 [D loss: 0.701871, acc.: 51.56%, op_acc: 100.00%] [G loss: 0.696091]
epoch:29 step:28095 [D 